Prerequisite installs for Project 2 - Quantum teleportation (Simulation)

In [ ]:
%pip install qiskit
%pip install qiskit[visualization]

Project 2 - Quantum teleportation (Simulation)

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram

# Creating registers
q = QuantumRegister(name='q', size=3)
c = ClassicalRegister(name='c', size=3)

# Step 0 - Instantiate qubit 0 in the state to be teleported:

teleport.barrier() # just a visual aid
# Step 1 - Make the shared entangled state in between qubit 1 and qubit 2

teleport.barrier() # just a visual aid
# Step 2 & 3: Alice applies a series of gates on qubit 0 and qubit 1

# Step 4 - Alice measures qubit 0 and qubit 1

# Step 5 - Bob applies a unitary transformation on his qubit
# based on the outcome of Alice's measurements

teleport.x(...).c_if(..., ...)
teleport.z(...).c_if(..., ...)
teleport.barrier() # just a visual aid

# Visualizing your teleportation circuit

teleport.draw()

# How can Bob quickly check if his qubit is actually in state |ϕ⟩=
# Implement your solution...
# Step 6 - Bob checks if he got the right state

#and then run a simulation:

backend = BasicSimulator()
result = backend.run(teleport, shots=8192).result()
counts = result.get_counts()
print(f"The counts for circuit teleport are: {counts}")
plot_histogram(counts)

Prerequisite installs for Project 2 - Quantum teleportation (IBM Quantum Platform)

In [ ]:
%pip install qiskit
%pip install qiskit[visualization]
%pip install qiskit-ibm-runtime

Save account to Qiskit, according to configuration file 'ibm_quantum_config.json'.
To get your IBM Quantum Platform token, go to https://quantum.ibm.com/account and copy your API token.
The 'ibm_quantum_config.json' file should contain something like this:

{
    "channel": "ibm_quantum",
    "token": "yourtoken"
}

In [ ]:
import os
import sys
import json
from qiskit_ibm_runtime import QiskitRuntimeService


if not os.path.exists("ibm_quantum_config.json"):
    print("Please create a file named 'ibm_quantum_config.json' with your IBM Quantum API token.")
    sys.exit(1)

with open("ibm_quantum_config.json", "r") as config_file:
    config = json.load(config_file)
    print("Configuration loaded successfully.")

QiskitRuntimeService.save_account(
    channel=config["channel"],
    token=config["token"],
    overwrite=True
)

print("Account saved successfully.")

Load account from Qiskit.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService


service = QiskitRuntimeService()
print("Account loaded successfully!")

backends = service.backends(
    filters=lambda b: not b.configuration().simulator and b.status().operational
)
print(f"Available Backends: {backends}")

least_busy_backend = service.least_busy(
    filters=lambda b: not b.configuration().simulator and b.status().operational
)
print(f"Least busy backend: {least_busy_backend.name}")

least_busy_backend_nb_qubits = least_busy_backend.configuration().n_qubits
print(f"Least busy backend number of qubits: {least_busy_backend_nb_qubits}")

Project 2 - Quantum teleportation (IBM Quantum Platform)

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram

# Creating registers
q = QuantumRegister(name='q', size=3)
c = ClassicalRegister(name='c', size=3)

# Step 0 - Instantiate qubit 0 in the state to be teleported:

teleport.barrier() # just a visual aid
# Step 1 - Make the shared entangled state in between qubit 1 and qubit 2

teleport.barrier() # just a visual aid
# Step 2 & 3: Alice applies a series of gates on qubit 0 and qubit 1

# Step 4 - Alice measures qubit 0 and qubit 1

# Step 5 - Bob applies a unitary transformation on his qubit
# based on the outcome of Alice's measurements

teleport.x(...).c_if(..., ...)
teleport.z(...).c_if(..., ...)
teleport.barrier() # just a visual aid

# Visualizing your teleportation circuit

teleport.draw()

# How can Bob quickly check if his qubit is actually in state |ϕ⟩=
# Implement your solution...
# Step 6 - Bob checks if he got the right state

#and then run a simulation:

backend = BasicSimulator()
result = backend.run(teleport, shots=8192).result()
counts = result.get_counts()
print(f"The counts for circuit teleport are: {counts}")
plot_histogram(counts)